In [6]:
import sys
sys.prefix

'/Users/xiaochenzhang/anaconda3/envs/python36'

In [7]:
!type pip

pip is /Users/xiaochenzhang/anaconda3/envs/python36/bin/pip


In [8]:
!pip install xgboost

In [32]:
import xgboost as xgb
import pandas as pd
import numpy as np
from tfidf import get_instance_tfidf_vector
import time
import csv

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [13]:
train_data = pd.read_csv('data/4w_trainset.csv', encoding='GB18030')
test_data = pd.read_csv('data/4k_testset.csv', encoding='GB18030')
train_data = train_data[:10000]
test_data = test_data[:4000]

In [16]:
train = pd.DataFrame({
    'ID': train_data['ID'],
    'Label': train_data[train_data.columns[-1]],
    'Feature': train_data['ID'].apply(lambda x: get_instance_tfidf_vector(str(x), False))
})

In [17]:
test = pd.DataFrame({
    'ID': test_data['ID'],
    'Label': test_data[test_data.columns[-1]],
    'Feature': test_data['ID'].apply(lambda x: get_instance_tfidf_vector(str(x), False))
})

In [ ]:
X_train = np.array(train['Feature'].values.tolist())
Y_train = train['Label']

In [23]:
X_true_test = np.array(test['Feature'].values.tolist())
Y_true_test = test['Label']

In [24]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.4, random_state=42)

In [26]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)
dtruetest = xgb.DMatrix(X_true_test, label=Y_true_test)

In [28]:
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 120
param['silent'] = 1
# param['nthread'] = 4
param['num_class'] = 5

num_round = 2

bst = xgb.train(param, dtrain, num_round)

In [30]:
ytrainpred = bst.predict(dtrain)
ytestpred = bst.predict(dtest)
ytruetestpred = bst.predict(dtruetest)

In [35]:
print('Train Acc: ', accuracy_score(ytrainpred, y_train))
print('Test Acc: ', accuracy_score(ytestpred, y_test))
print('True Test Acc: ', accuracy_score(ytruetestpred, Y_true_test))

Train Acc:  0.8873333333333333
Test Acc:  0.80925
True Test Acc:  0.2455
